### Imports

In [2]:
import pandas as pd

## Inputs

In [1]:
filepath = 'data/IFC-POWER_BI.csv'

In [11]:
df = pd.read_csv(filepath, sep=';')
df.head();

C:\Users\dgm\AppData\Local\Temp\ipykernel_32732\1714439586.py:1: DtypeWarning: Columns (2,3,4,5,6,7,17,18,19,20,32,34,35,36,37,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, sep=';')


In [10]:
df.dtypes

Model                               object
GUID                                object
ExtrusionX (mm)                     object
ExtrusionY (mm)                     object
ExtrusionZ (mm)                     object
OriginX (mm)                        object
OriginY (mm)                        object
OriginZ (mm)                        object
XDirX                              float64
XDirY                              float64
XDirZ                              float64
Last Modified Date                  object
Product Object Type                 object
Change Action                       object
Owning User                         object
Product Description                 object
Product Name                        object
CenterOfGravityX (mm)               object
CenterOfGravityY (mm)               object
CenterOfGravityZ (mm)               object
Volume (m³)                         object
NAME                                object
ASSEMBLY.ID                        float64
CONTENTTYPE

In [ ]:
decimal_cols = ('ExtrusionX (mm)', 'ExtrusionY (mm)', 'ExtrusionZ (mm)', 'OriginX (mm)', 'OriginY (mm)', 'OriginZ (mm)',
'XDirX', 'XDirY', 'XDirZ', 'CenterOfGravityX (mm)', 'CenterOfGravityY (mm)', 'CenterOfGravityZ (mm)',
'Volume (m³)', 'LENGTH (mm)', 'VOLUME (m³)', 'HEIGHT (mm)', 'AREA_PGZ (m²)', 'AREA (m²)', 'WIDTH (mm)', 'WEIGHT (t)', 'PROFILE.CROSS_SECTION_AREA (m²)')
for col in decimal_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')